# Testing calculating distances and angles

Goal: the distance between my given coordinate and the closest sea ice concentration above threshold, in a certain wind direction.

method:
1. select out only values of the field that is around the threshold I am interested in (limits datapoints co calculate)
2. calculate the angle between the observation coordinate and all of the selected values
3. select out only the angles that are within the wind field for the flight
4. calculate the distance
5. select the smallest distance


In [1]:
# ---Packages---
import xarray as xr
import numpy as np
import pandas as pd


In [2]:
# original coordinates of point(observation)

# test coordinates IS22-02
#lat_orig=70.2
#lon_orig=20.38

# test coordinates IS22-08
lat_orig=71.82
lon_orig=19.74

In [16]:
# Read in sea ice data for testing

# This date corresponds to the first islas campaign day. That day had a main wind direction towards the west
# which I think mean that the sea ice is outside of our area of interest. work on the other dates first.
#date = '20220322' 

# IS22-08
date = '20220330'


sic_ds = xr.open_dataset('/home/ninalar/Documents/MC2/procIslasMicrophy/sea_ice_satellite/asi-n6250-' + date + '-5.4_regridded.nc')
sic_ds.close()

# rename data variable and update attributes
sic_ds['sic'] = sic_ds['__xarray_dataarray_variable__'].assign_attrs(units="Percent", description="Sea Ice Concentration")
sic_ds = sic_ds.drop_vars(['__xarray_dataarray_variable__'])

sic_ds
sic_df = sic_ds.to_dataframe()


# select only values where sic is >25 or less than 26
sic_sel_df = sic_df[(sic_df['sic'] >= 25) & (sic_df['sic'] < 50)]

sic_sel_df = sic_sel_df.reset_index(level=[0,1]) # reindex to drop multiindex
sic_sel_df

,lat,lon,sic
0,42.125,139.875,49.188972
1,45.375,144.625,29.353834
2,46.125,150.125,25.804480
3,46.375,299.625,25.004175
4,46.875,297.875,29.893982
...,...,...,...
1055,82.625,50.375,47.885918
1056,82.625,50.625,35.637527
1057,82.625,51.375,38.106197
1058,82.625,51.625,39.154926


In [24]:
# Calculate direction with atan2(y2-y1,x2-x1)

sic_sel_df['angle'] = np.atan2(sic_sel_df['lon']-lon_orig, sic_sel_df['lat']-lat_orig)*(180/np.pi)
sic_sel_df['angle'].where(sic_sel_df['angle']>=0,sic_sel_df['angle']+360) # adjust any negative values (atan2 output is from -pi to pi)
sic_sel_df


# Select out only the values that are within the wind directions for the flight (goal winddirection: 281)
wd_min=93
wd_max=98
sic_sel_df = sic_sel_df[(sic_sel_df['angle'] >= wd_min) & (sic_sel_df['angle'] < wd_max)]

sic_sel_df

#sic_ds['angle']= np.atan2(sic_ds.lon-lon_orig, sic_ds.lat-lat_orig)*(180/np.pi) # get angle in degrees
# atan 2 output is in the range -pi to pi and negative values must be adjusted (add 360)
#sic_ds['angle']=sic_ds['angle'].where(sic_ds['angle']>=0,sic_ds['angle']+360)

#sic_ds
# Select out only the values that are within the wind directions for the flight (goal winddirection: 281)
#wd_min=275
#wd_max=285

#sic_wd = sic25.where((sic25.angle>=wd_min) & (sic25.angle<wd_max),drop=True)

,lat,lon,sic,angle
3,46.375,299.625,25.004175,95.194613
4,46.875,297.875,29.893982,95.124955
5,46.875,298.125,26.568113,95.120377
7,47.125,298.125,43.159660,95.069330
8,47.125,298.875,45.826866,95.055780
...,...,...,...,...
524,69.125,47.125,25.593454,95.620468
525,69.125,47.625,41.105015,95.520317
528,69.375,47.875,31.176752,94.966665
563,71.125,27.375,34.815777,95.201195


In [6]:
# Calculate distance with Haversine formula
sic_wd

<xarray.Dataset> Size: 0B
Dimensions:  (lat: 0, lon: 0)
Coordinates:
  * lat      (lat) float64 0B 
  * lon      (lon) float64 0B 
Data variables:
    sic      (lat, lon) float32 0B 
    angle    (lon, lat) float64 0B